# Problem Statement
### Where will a new guest book their first travel experience?

# Dataset Description
In this challenge, we are given a list of users along with their demographics, web session records, and some summary statistics. We are asked to predict which country a new user's first booking destination will be. All the users in this dataset are from the USA.

There are 12 possible outcomes of the destination country: 'US', 'FR', 'CA', 'GB', 'ES', 'IT', 'PT', 'NL','DE', 'AU', 'NDF' (no destination found), and 'other'.

### File descriptions

- train_users.csv - the training set of users


- test_users.csv - the test set of users 
`id: user id 
date_account_created: the date of account creation
timestamp_first_active: timestamp of the first activity, note that it can be earlier than date_account_created or  date_first_booking because a user can search before signing up 
date_first_booking: date of first booking 
gender 
age 
signup_method 
signup_flow: the page a user came to signup up from 
language: international language preference 
affiliate_channel: what kind of paid marketing 
affiliate_provider: where the marketing is e.g. google, craigslist, other 
first_affiliate_tracked: whats the first marketing the user interacted with before the signing up 
signup_app 
first_device_type 
first_browser 
country_destination: this is the target variable you are to predict `


- sessions.csv - web sessions log for users 
`user_id: to be joined with the column 'id' in users table 
action 
action_type 
action_detail 
device_type 
secs_elapsed `


- countries.csv - summary statistics of destination countries in this dataset and their locations 


- age_gender_bkts.csv - summary statistics of users' age group, gender, country of destination 


# Evalutaion Criterion
![Evaluation Criterion](evaluation.png)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import scipy.stats as stats

# Disable warnings from printing
from warnings import filterwarnings
filterwarnings('ignore')

from sklearn.neighbors import KNeighborsClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.preprocessing import LabelEncoder

# Draw inline
%matplotlib inline

/home/rehas/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## User Data Processing

In [2]:
# Read the data
train_users = pd.read_csv('../Data/train_users.csv')
test_users = pd.read_csv('../Data/test_users.csv')

# Drop country for processing
LABEL = 'country_destination'
labels = train_users[LABEL].copy()
train_users.drop(LABEL, inplace=True, axis=1)

# Merge train and test users
users = pd.concat((train_users, test_users), axis=0, ignore_index=True)

# Drop id for processing
ids = users["id"].copy()
users.drop("id", inplace=True, axis=1)

### Null values

Creating a new column which counts the number of null values in each record helps to distinguish '-1' as a special type of value (missing) compared to other values.

In [3]:
# Fill null with -1 for missing values
users.fillna(-1, inplace=True)

### Date Parsing

- Dates cannot be used as such. They are parsed to extract date of the weekday, week number, month, month and year.
- This is done for date_account_created and timestamp_first_active.
- There is another date feature date_first_booking which is completely null for test users. So we can ignore it completely.
- Also compute the difference between dates date_account_created and timestamp_first_active.

In [4]:
# timestamp_first_active pre-processing to get date
users.timestamp_first_active = users.timestamp_first_active.apply(str)
users.timestamp_first_active = users.timestamp_first_active.str[:8]

# Convert strings to dates
users.date_account_created = pd.to_datetime(users.date_account_created, format="%Y-%m-%d")
users.timestamp_first_active = pd.to_datetime(users.timestamp_first_active, format="%Y%m%d")

def _parse_date(time_dt):
    return [time_dt.year, time_dt.month, time_dt.day, time_dt.weekday(), time_dt.isocalendar()[1]]

def extract_dates_inplace(features, date_column):
    extracted_vals = np.vstack(features[date_column].apply(
        (lambda x: _parse_date(x))))
    for i, period in enumerate(['year', 'month', 'day', 'weekday', 'weekno']):
        features['%s_%s' % (date_column, period)] = extracted_vals[:, i]
    features.drop(date_column, inplace=True, axis=1)

# date_account_created processing
extract_dates_inplace(users, "date_account_created")
extract_dates_inplace(users, "timestamp_first_active")

# Since date_first_booking is null for entire test data, we can simply drop it.
users.drop(['date_first_booking'], inplace=True, axis=1)

### Age
- Age has lot of missing values.
- It also has lot of spurious values like < 14 or > 100.
- Replacing all these with -1.
- We can leave these as -1 or predict all the -1 age values using a classifier such as KNN based on other columns.
- Age values < 2000 and > 1900 are very likely to be birth years. so estimate age by 2014 - age.

In [5]:
# Age is missing for values -1, and spurious values > 100 or < 14
users.age[users.age > 100] = -1
users.age[users.age < 14] = -1

### Categorical Features
- affiliate_channel, affiliate_provider, first_affiliate_tracked, first_browser,                    first_device_type, gender, language, signup_app, signup_method, signup_flow are all the categorical features. They also contain missing values which have been replace by -1.
- Next one hot encoding is carried out on each of them.

In [6]:
CATEGORICAL_FEATURES = ['affiliate_channel', 'affiliate_provider',
                        'first_affiliate_tracked', 'first_browser',
                        'first_device_type', 'gender', 'language', 'signup_app',
                        'signup_method', 'signup_flow']

for feature in CATEGORICAL_FEATURES:
    users = pd.concat((users, pd.get_dummies(users[feature], prefix=feature)), axis=1)
    users = users.drop(feature, axis=1)

In [81]:
# Save processed training and testing data.
training = users[:train_users.shape[0]]
testing = users[train_users.shape[0]:].reset_index().drop("index", axis=1)

# Model
Below is the implementation of a simple XGBoost model that predicts 5 highest likely countries for each user in the test dataset. This is done because of the evaluation metric.

In [82]:
np.random.seed(0)
le = LabelEncoder()
xgb = XGBClassifier(max_depth=6, learning_rate=0.3, n_estimators=25,
                    objective='multi:softprob', subsample=0.5, colsample_bytree=0.5, seed=0)                  
xgb.fit(training, le.fit_transform(labels))
predictions = xgb.predict_proba(testing)

In [83]:
#Taking the 5 classes with highest probabilities
test_ids = ids[train_users.shape[0]:].reset_index().drop("index", axis=1)
ids_result = []  #list of ids
cts_results = []  #list of countries
for i in range(len(test_ids)):
    idx = test_ids.iloc[i]
    ids_result += [idx] * 5
    cts_results += le.inverse_transform(np.argsort(predictions[i])[::-1])[:5].tolist()

#Generate submission
submission = pd.DataFrame(np.column_stack((ids_result, cts_results)), columns=['id', 'country'])
submission.to_csv('../Data/submission.csv',index=False)

# Results
With above minimal feature engineering and XGBoost model we get an NDCG@5 score of 0.86995 and a Kaggle ranking of 537/1500 on the private leaderboard.

# Sessions Data Processing

In [7]:
# Read the sessions data
sessions_data = pd.read_csv("../Data/sessions.csv")

In [8]:
# Get unique user ids to store results
sessions_data_ids = sessions_data.loc[:, ["user_id"]].drop_duplicates().reset_index().drop("index", axis=1)

First we count number of sessions by each user

In [9]:
# Get number of sessions of each user
sessions_data_ids = sessions_data_ids.join(
    sessions_data.groupby(['user_id'])["user_id"].size().to_frame(),
    on="user_id")
sessions_data_ids.rename(columns={0 : "NumSessions"}, inplace=True)

### Action, action_type, action_detail and device_type processing
- Next for each of the attributes action, action_type, action_detail and device_type, we want to see what kind of distributions they have for each user.
- Specifically we count the different types of actions for each user and also compute mean and standard deviation of this distribution of counts.

In [10]:
# Get user ids and actions for action processing
sessions_data_action = sessions_data.loc[:, ["user_id", "action"]]

# Processing for Action attribute
# Replace null values with 'NAN'
sessions_data_action.action = sessions_data_action.action.fillna("NAN")

# Replace infrequent values with 'Other' to avoid overfitting and speed up computation
THRESHOLD = 0.0005*sessions_data_action.shape[0]
counts = sessions_data_action.action.value_counts()
sessions_data_action.action = sessions_data_action.action.apply(lambda x: 'Other' if counts[x] < THRESHOLD else x)

# Get counts of each action for each user
unique_actions = sessions_data_action["action"].unique()

for act in unique_actions:
    sessions_data_ids = sessions_data_ids.join(
        sessions_data_action.loc[sessions_data_action.action == act, :].groupby(['user_id']).size().to_frame(),
        on="user_id")
    sessions_data_ids.rename(columns={0 : "action_" + act}, inplace=True)
    
sessions_data_ids = sessions_data_ids.fillna(0)

# Get mean and std of distribution of counts of actions for each user
sessions_data_ids["NumActionsMean"] = \
sessions_data_ids.loc[:, "action_lookup":"action_similar_listings_v2"].mean(axis=1)

sessions_data_ids["NumActionsStd"] = \
sessions_data_ids.loc[:, "action_lookup":"action_similar_listings_v2"].std(axis=1)

In [11]:
# Get user ids and action_type for action_type processing
sessions_data_action_type = sessions_data.loc[:, ["user_id", "action_type"]]

# Processing for action_type attribute
# Replace null values with 'NAN'
sessions_data_action_type.action_type = sessions_data_action_type.action_type.fillna("NAN")
sessions_data_action_type.action_type = sessions_data_action_type.action_type.replace(
    {
        '-unknown-': 'NAN'
    }
)

# Replace infrequent values with 'Other' to avoid overfitting and speed up computation
THRESHOLD = 0.005*sessions_data_action_type.shape[0]
counts = sessions_data_action_type.action_type.value_counts()
sessions_data_action_type.action_type = \
sessions_data_action_type.action_type.apply(lambda x: 'Other' if counts[x] < THRESHOLD else x)

# Get counts of each action_type for each user
unique_actions = sessions_data_action_type["action_type"].unique()

for act in unique_actions:
    sessions_data_ids = sessions_data_ids.join(
        sessions_data_action_type.loc[sessions_data_action_type.action_type == act, :].groupby(
            ['user_id']).size().to_frame(),
        on="user_id")
    sessions_data_ids.rename(columns={0 : "action_type_" + act}, inplace=True)
    
sessions_data_ids = sessions_data_ids.fillna(0)

# Get mean and std of distribution of counts of action_type for each user
sessions_data_ids["NumActionTypeMean"] = \
sessions_data_ids.loc[:, "action_type_NAN":"action_type_Other"].mean(axis=1)

sessions_data_ids["NumActionTypeStd"] = \
sessions_data_ids.loc[:, "action_type_NAN":"action_type_Other"].std(axis=1)

In [12]:
# Repeat procedure for action_detail
# Get user ids and action_detail for action_detail processing
sessions_data_action_detail = sessions_data.loc[:, ["user_id", "action_detail"]]

# Processing for action_detail attribute
# Replace null values with 'NAN'
sessions_data_action_detail.action_detail = sessions_data_action_detail.action_detail.fillna("NAN")
sessions_data_action_detail.action_detail = sessions_data_action_detail.action_detail.replace(
    {
        '-unknown-': 'NAN'
    }
)

# Replace infrequent values with 'Other' to avoid overfitting and speed up computation
THRESHOLD = 0.005*sessions_data_action_detail.shape[0]
counts = sessions_data_action_detail.action_detail.value_counts()
sessions_data_action_detail.action_detail = \
sessions_data_action_detail.action_detail.apply(lambda x: 'Other' if counts[x] < THRESHOLD else x)

# Get counts of each action_type for each user
unique_actions = sessions_data_action_detail["action_detail"].unique()

for act in unique_actions:
    sessions_data_ids = sessions_data_ids.join(
        sessions_data_action_detail.loc[sessions_data_action_detail.action_detail == act, :].groupby(
            ['user_id']).size().to_frame(),
        on="user_id")
    sessions_data_ids.rename(columns={0 : "action_detail_" + act}, inplace=True)
    
sessions_data_ids = sessions_data_ids.fillna(0)

# Get mean and std of distribution of counts of action_detail for each user
sessions_data_ids["NumActionDetailMean"] = \
sessions_data_ids.loc[:, "action_detail_NAN":"action_detail_listing_reviews"].mean(axis=1)

sessions_data_ids["NumActionDetailStd"] = \
sessions_data_ids.loc[:, "action_detail_NAN":"action_detail_listing_reviews"].std(axis=1)

In [13]:
# Repeat the procedure for device_type
# Get user ids and device_type for device_type processing
sessions_data_device_type = sessions_data.loc[:, ["user_id", "device_type"]]

# Processing for device_type attribute
# Replace null values with 'NAN'
sessions_data_device_type.device_type = sessions_data_device_type.device_type.fillna("NAN")
sessions_data_device_type.device_type = sessions_data_device_type.device_type.replace(
    {
        '-unknown-': 'NAN'
    }
)

# Replace infrequent values with 'Other' to avoid overfitting and speed up computation
THRESHOLD = 0.005*sessions_data_device_type.shape[0]
counts = sessions_data_device_type.device_type.value_counts()
sessions_data_device_type.device_type = \
sessions_data_device_type.device_type.apply(lambda x: 'Other' if counts[x] < THRESHOLD else x)

# Get counts of each action_type for each user
unique_actions = sessions_data_device_type["device_type"].unique()

for act in unique_actions:
    sessions_data_ids = sessions_data_ids.join(
        sessions_data_device_type.loc[sessions_data_device_type.device_type == act, :].groupby(
            ['user_id']).size().to_frame(),
        on="user_id")
    sessions_data_ids.rename(columns={0 : "device_type_" + act}, inplace=True)
    
sessions_data_ids = sessions_data_ids.fillna(0)

# Get mean and std of distribution of counts of device_type for each user
sessions_data_ids["NumDeviceTypeMean"] = \
sessions_data_ids.loc[:, "device_type_Windows Desktop":"device_type_Tablet"].mean(axis=1)

sessions_data_ids["NumDeviceTypeStd"] = \
sessions_data_ids.loc[:, "device_type_Windows Desktop":"device_type_Tablet"].std(axis=1)

### Processing for secs_elapsed
- secs_elapsed is a numerical attribute (time in seconds for that session).
- For each user we compute the average, standard deviation, median and skew of this distribution of secs_elapsed.

In [14]:
# Get user ids and secs_elapsed for secs_elapsed processing
sessions_data_secs_elapsed = sessions_data.loc[:, ["user_id", "secs_elapsed"]]
sessions_data_secs_elapsed.secs_elapsed = sessions_data_secs_elapsed.secs_elapsed.fillna(0)

# Get simple stats on secs_elapsed
tmp = sessions_data_secs_elapsed.groupby('user_id').aggregate(
    [
        np.mean, np.std, np.median, stats.skew
    ]
)

tmp.columns = ["secs_elapsed_mean", "secs_elapsed_std", "secs_elapsed_median", "secs_elapsed_skew"]

sessions_data_ids = sessions_data_ids.join(tmp, on="user_id")

In [15]:
# Free memory
del sessions_data_action
del sessions_data_action_detail
del sessions_data_action_type
del sessions_data_device_type
del sessions_data_secs_elapsed
del sessions_data

### Null values
- There are some users about whom we have no sessions information.
- Joing the two datasets will give Null values for sessions attributes for these users.
- So for each user, we keep track of total number of null values.

In [16]:
# combine ids back with users data
users["user_id"] = ids

# Merging train-test with session data
all_data = pd.merge(users, sessions_data_ids, how='left')
all_data = all_data.drop(['user_id'], axis=1)

#Missing features for samples without sesssion data.
all_data = all_data.fillna(-2)

# Count of all types of null 
all_data['NumAllNulls'] = (all_data < 0).astype(int).sum(axis=1)

In [17]:
# Save processed training and testing data.
training = all_data[:train_users.shape[0]]
testing = all_data[train_users.shape[0]:].reset_index().drop("index", axis=1)

# Model
Below is the implementation of a simple XGBoost model that predicts 5 highest likely countries for each user in the test dataset. This is done because of the evaluation metric.

In [18]:
np.random.seed(0)
le = LabelEncoder()
xgb = XGBClassifier(max_depth=6, learning_rate=0.3, n_estimators=25,
                    objective='multi:softprob', subsample=0.5, colsample_bytree=0.5, seed=0)                  
xgb.fit(training, le.fit_transform(labels))
predictions = xgb.predict_proba(testing)

In [19]:
#Taking the 5 classes with highest probabilities
test_ids = ids[train_users.shape[0]:].reset_index().drop("index", axis=1)
ids_result = []  #list of ids
cts_results = []  #list of countries
for i in range(len(test_ids)):
    idx = test_ids.iloc[i]
    ids_result += [idx] * 5
    cts_results += le.inverse_transform(np.argsort(predictions[i])[::-1])[:5].tolist()

#Generate submission
submission = pd.DataFrame(np.column_stack((ids_result, cts_results)), columns=['id', 'country'])
submission.to_csv('../Data/submission.csv',index=False)

# Results
Above Users + sessions + xgboost model gives us an NDCG@5 score of 	0.88324 and a Kaggle ranking of 232/1500 on the private leaderboard.